In [1]:
import pandas as pd
import os
import dataframe_image as dfi 

pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:,.4f}'.format

In [2]:
base_dir = '../..'
helper_dir = os.path.join(base_dir, 'helper')
output_dir = os.path.join(base_dir, 'output')
png_dir = os.path.join(output_dir, 'png')
save_fig_dir = os.path.join(png_dir, 'FA')
os.makedirs(save_fig_dir, exist_ok=True)

%run {helper_dir}/defillama.py

In [3]:
dllama = DefiLlama()

## Obtain basic metrics for fundamental analysis

Instead of going to multiple websites (coingecko, defillama, GMX official site) to look up the basic metrics, we can get them programmatically. 

In [4]:
df = dllama.get_protocols_fundamentals()
da = df[df.symbol == 'GMX']
tvl, mcap, fdv = da.tvl.iloc[0], da.mcap.iloc[0], da.fdv.iloc[0]

In [5]:
dd = {'gmx':'coingecko',                                        # GMX on coingecko
      '0xfc5a1a6eb076a2c7ad06ed22c90d7e710e35ad0a':'arbitrum',  # GMX on arbitrum
      '0x62edc0692BD897D2295872a9FFCac5425011c661':'avax',      # GMX on avalanche
      }
prices = dllama.get_tokens_curr_prices(dd)
# prices

In [6]:
price = prices.price.mean()

In [7]:
circ_supply = mcap / price
max_supply = fdv / price
pct_circulating = mcap / fdv

In [8]:
print('TVL:', "${:,.0f}".format(tvl))
print('Mcap:', "${:,.0f}".format(mcap))
print('FDV:', "${:,.0f}".format(fdv))
print('Price:', "${:.2f}".format(price))
print('Circulating Supply:', "{:,.0f}".format(circ_supply))
print('Max Supply:', "{:,.0f}".format(max_supply))
print('Circulating Supply (%):', "{:.0%}".format(pct_circulating))

TVL: $450,533,005
Mcap: $340,189,303
FDV: $558,653,036
Price: $42.43
Circulating Supply: 8,017,660
Max Supply: 13,166,463
Circulating Supply (%): 61%


## Is the current price justified?

In [9]:
print('{:.0%} tokens yet to enter the market'.format(1-pct_circulating))

39% tokens yet to enter the market


In [10]:
# From GMX doc:
#
# The increase in circulating supply will vary depending on the number of tokens that get vested, 
# and the amount of tokens used for marketing / partnerships.
#
# 1 million GMX tokens reserved for marketing, partnerships and community developers.
# 250,000 GMX tokens distributed to contributors linearly over 2 years.
#
# current esGMX emission schedule: https://snapshot.org/#/gmx.eth/proposal/0xb370249628b2226c6a7e771b2959c3b2e80eada36ad3618a7fc39f964213643e
# it takes 1 year for 1 esGMX to become 1 GMX
#
# Minting beyond the max supply of 13.25 million is controlled by a 28 day timelock. 
# This option will only be used if more products are launched and liquidity mining is required, a governance vote 
# will be conducted before any changes.

In [11]:
def calc_required_growth_next_year(marketing_partner_dev_release=1e6, new_mint=2e6):
    # Calculate the growth of protocol required for the next year to maintain current price.
    # 
    # marketing_partner_dev_release: number of tokens to be released for marketing, partnerships and community 
    #   devs during next year. A total of 1 million tokens was reserved for these purposes at 
    #   birth of protocol, and some are probably already released.
    #
    # new_mint: number of tokens to be minted, beyond the current max supply of 13.25 million, to support new products
    
    # roughly 50% of team tokens will unlock in next year since protocol started between June and Sept 2021
    team_vested = 250_000 / 2
    
    # calculate roughly the amount of esGMX rewards that will become GMX according to esGMX emission schedule
    # https://snapshot.org/#/gmx.eth/proposal/0xb370249628b2226c6a7e771b2959c3b2e80eada36ad3618a7fc39f964213643e
    esGMX_vested = 250_000 + 175_000 + 150_000 + 150_000 + 100_000 
                       
    future_circ_supply = circ_supply + team_vested + esGMX_vested + marketing_partner_dev_release  + new_mint
    future_max_supply = max_supply + new_mint

    return {'required growth %':price * future_circ_supply / mcap - 1, 'circulating supply %':future_circ_supply / future_max_supply}

In [12]:
df = pd.DataFrame([calc_required_growth_next_year(1e6, 4e6), 
                   calc_required_growth_next_year(1e6, 2e6), 
                   calc_required_growth_next_year(1e6, 1e6), 
                   calc_required_growth_next_year(1e6, 0), 
                   calc_required_growth_next_year(.5e6, 0)])

df['biz dev spend qty'] = [1e6, 1e6, 1e6, 1e6, .5e6]
df['new mint qty'] = [4e6, 2e6, 1e6, 0, 0]
df['scenario'] = ['wasteful expansion', 'prudent expansion', 'prudent expansion', 'resourceful expansion', 'status quo']

df = df.loc[:, ['scenario', 'biz dev spend qty', 'new mint qty', 'required growth %', 'circulating supply %']]
df = df.set_index('scenario')

In [13]:
format_dict = {'biz dev spend qty': '{:,.0f}',
               'new mint qty': '{:,.0f}',
               'required growth %': '{:.0%}', 'circulating supply %': '{:.1%}'}
df_styled = df.style.format(format_dict)
dfi.export(df_styled, os.path.join(save_fig_dir, 'required_growth_scenarios.png'))

/usr/local/lib/python3.8/site-packages/dataframe_image/_pandas_accessor.py:69: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  html = '<div>' + obj.render() + '</div>'
[0927/140212.739018:INFO:headless_shell.cc(660)] Written to file /var/folders/qq/v47zfw7s1kn38gw67q4v194r0000gn/T/tmp_a6at1go/temp.png.
